#### CVEN 6525 - NONLINEAR ANALYSIS OF FRAMED STRUCTURES
##### FINAL TERM PROJECT - SPRING 2024 - RON BAILIE, UNIVERSITY OF COLORADO BOULDER

<center> Topology Optimization of 2D structural elements using the SIMP method</center><br>  
The following is a python implementation of Ole Sigmund's <i>a 99 line topology optimization code written in MATLAB</i>